In [16]:
from read_data import get_training, get_test, get_sparse
from processing import exclude_non_numeric
from utils import create_csv_output
import numpy as np
import pandas as pd
import scipy
import pickle
from scipy.sparse import csr_matrix
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, FunctionTransformer
from sklearn.model_selection import train_test_split, cross_val_score, ShuffleSplit, StratifiedShuffleSplit
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
np.random.seed(30027)
sns.set(rc={"figure.facecolor": "white"})

In [17]:
from sklearn.neural_network import MLPClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import Lasso

## Train data

In [18]:
train_X,train_y = get_training()
train_X_numeric = csr_matrix(FunctionTransformer(np.log1p).fit_transform(exclude_non_numeric(train_X)))
train_name, train_ingr, train_steps = get_sparse(data="train")
train_sparse = scipy.sparse.hstack((train_X_numeric, train_name, train_ingr, train_steps), format="csr")

In [19]:
# Change target from (1.0, 2.0, 3.0) -> (0, 1, 2)
y = (train_y - 1).astype(int)
X_train, X_test, y_train, y_test = train_test_split(train_sparse, y, test_size=0.2)

## Test data

In [20]:
test_X = get_test()
test_X_numeric = csr_matrix(FunctionTransformer(np.log1p).fit_transform(exclude_non_numeric(test_X)))
test_name, test_ingr, test_steps = get_sparse(data="test")
test_sparse = scipy.sparse.hstack((test_X_numeric, test_name, test_ingr, test_steps), format="csr")

## MLPClassifier

In [21]:
MLP_pipeline = make_pipeline(
    StandardScaler(with_mean=False),
    SelectFromModel(Lasso(alpha=0.01)),
    MLPClassifier()
)

In [7]:
%%time
partial_MLP = MLP_pipeline.fit(X_train, y_train)
pickle.dump(partial_MLP, open("report_models/partial/MLP.sav", "wb"))
# CPU times: user 16min 34s, sys: 3min 19s, total: 19min 53s
# Wall time: 6min 29s

In [ ]:
cross_val_score(partial_MLP, X_test, y_test, cv=StratifiedShuffleSplit(n_splits=5, test_size=0.2))
# array([0.738125, 0.700625, 0.700625, 0.689375, 0.7125  ])

### Analysis

In [ ]:
partial_MLP["selectfrommodel"].transform(X_train)

In [ ]:
MLP_score = partial_MLP.score(X_test, y_test)
MLP_pred = partial_MLP.predict(X_test)
MLP_cm = confusion_matrix(y_test, MLP_pred, normalize='true')

In [ ]:
plt.figure(figsize=(5, 5))
sns.heatmap(MLP_cm, annot=True, square=True, xticklabels=[1.0, 2.0, 3.0], yticklabels=[1.0, 2.0, 3.0], cmap="Blues_r")
plt.ylabel('Actual label')
plt.xlabel('Predicted label' + '\n\nAccuracy Score: {0}'.format(MLP_score))
plt.title('MLPClassifier Confusion Matrix', size=12)
plt.savefig("report_pics/NeuralNetwork/MLP_cm.png")

### Submission model

In [23]:
%%time
MLP = MLP_pipeline.fit(train_sparse, y)
pickle.dump(MLP, open("report_models/submission/MLP.sav", "wb"))


CPU times: user 5min 32s, sys: 1min 47s, total: 7min 19s
Wall time: 1min 50s
/Users/mcken/.anyenv/envs/pyenv/versions/3.8.5/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [24]:
create_csv_output("report_submission/MLP", (MLP.predict(test_sparse) + 1).astype(float))